In [ ]:
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np

# Don't collapse Pandas Dataframes:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
def normalize(predicted_ratings):
    normalized_predicted_ratings = (predicted_ratings - predicted_ratings.min()) / (predicted_ratings.max() - predicted_ratings.min())
    return normalized_predicted_ratings
  
def generate_prediction_df(values, df, n_factors):
    ## Matrix Factorization
    u, s, v = svds(values, k = n_factors)
    s = np.diag(s)

    ## Calculate Predicted User Ratings
    pred_ratings = np.dot(np.dot(u, s), v) 
    pred_ratings = normalize(pred_ratings)
    predictions_df = pd.DataFrame(pred_ratings,
                                  columns = df.columns,
                                  index = list(df.index)).transpose()
    return predictions_df

def recommend_items(predictions_df, user_id, number_of_recommendations):
    user_predictions_df = predictions_df[user_id].sort_values(ascending = False).reset_index().rename(columns = {user_id : 'Similarity Score'})
    user_recommendations_df = user_predictions_df.sort_values(by='Similarity Score', ascending = False).head(number_of_recommendations)
    return user_recommendations_df


In [ ]:
## Import Dataset
df = pd.read_csv('dataset.csv')
print('Dataset Shape: {0}'.format(df.shape))
df.sort_values(['user_id'], ascending=False).head(10)

In [ ]:
## Generate a pivot table
pivot_df = df.pivot_table(columns = 'book_id', 
                            index = 'user_id', 
                           values = 'book_rating').fillna(0)
pivot_df.head()

In [ ]:
## Build csr matrix: (compressed sparse row)
df_values = pivot_df.values
values = csr_matrix(df_values)
n_factors = 10 # Number of singular values and vectors to compute
predictions_df = generate_prediction_df(values, pivot_df, n_factors)
predictions_df.head()

In [ ]:
## Generate user recommendations
user_id = 1001
number_of_recommendations = 10
user_recommendations = recommend_items(predictions_df, user_id, number_of_recommendations)
user_recommendations

In [ ]:
## Generate user recommendations
user_id = 24854
number_of_recommendations = 10
user_recommendations = recommend_items(predictions_df, user_id, number_of_recommendations)
user_recommendations

In [ ]:
## Generate user recommendations
user_id = 23112
number_of_recommendations = 10
user_recommendations = recommend_items(predictions_df, user_id, number_of_recommendations)
user_recommendations

In [ ]:
## Generate user recommendations
user_id = 4987
number_of_recommendations = 10
user_recommendations = recommend_items(predictions_df, user_id, number_of_recommendations)
user_recommendations